In [294]:
import pandas as pd
import numpy as np
import re
import os
import pdfplumber
import glob
import pdfplumber
import os

In [295]:
location='path/Banks_Statement/'

In [296]:
bank_statement_pdf=[]
for foldername,subfolders,files in os.walk(location):
    for file in files:
        if file.endswith('.pdf'):
            file1=os.path.join(foldername,file)
            bank_statement_pdf.append(file1)

In [297]:
len(bank_statement_pdf)

753

In [1]:
#bank_statement_pdf[:5]

In [2]:
#bank_statement_pdf[1][-55:]

In [300]:
final_df = pd.DataFrame()

for file in bank_statement_pdf:
    all_text = ''
    with pdfplumber.open(file) as pdf:
                for pdf_page in pdf.pages:
                    single_page_text = pdf_page.extract_text()
                    all_text = all_text + '\n' + single_page_text
    

    def is_double_date(x):
        import re
        match = re.findall(r'(\d+/\d+/\d+) (\d+/\d+/\d+)',x)
        return match 
    
    ans=[x for x in all_text.split("\n") if is_double_date(x)]
    #print(ans)
    df=pd.DataFrame(columns=['Tran. Date','Value Date','Customer Ref.','Bank Ref.' ,'Credit Amount', 'Balance'])
    for i,j in enumerate(ans):
        x = [x for x in j.split(" ") if x]
        x = x[0:2] + x[-4:]
        df.loc[i]=x
    df=df.reset_index(drop=True)
    df['Tran. Date']=pd.to_datetime(df['Tran. Date'])
    df['Value Date']=pd.to_datetime(df['Value Date'])
    #df['Credit Amount'] = df['Credit Amount'].str.replace(',',"").astype(float)
    df['Balance'] = df['Balance'].str.replace(',',"",regex=True).str.replace('\(',"",regex=True).str.replace('\)',"",regex=True).astype(float)
    df['Filename'] = file[-55:]
    final_df = pd.concat([final_df,df],axis = 0)

In [301]:
bank_statement_data=final_df.copy(deep=True)
bank_statement_data.shape

(3363, 7)

In [4]:
#bank_statement_data.head()